Dependency installation

In [ ]:
!pip install -q sentence-transformers google-generativeai pandas numpy

Read bembedding

In [ ]:
import numpy as np, pandas as pd
E = np.load("embeddings.npy")


using sentence_transformaers to make query embedding

In [ ]:
from sentence_transformers import SentenceTransformer
model_name = "all-MiniLM-L6-v2"
st_model = SentenceTransformer(model_name, device="cpu")

def embed_query(text: str) -> np.ndarray:
    return st_model.encode(
        [text],
        convert_to_numpy=True,
        normalize_embeddings=True,
    )[0]  # (d,)

def search(query: str, top_k: int = 5):
    q = embed_query(query)           
    scores = E @ q                   
    idxs = np.argsort(-scores)[:top_k]

    results = []
    for i in idxs:
        results.append({
            "score": float(scores[i]),
            "idx": int(i),
            "complaint_id": df.loc[i, "complaint_id"] if "complaint_id" in df.columns else None,
            "text": df.loc[i, "text"],
        })
    return results


Taking gemini key

In [ ]:
import google.generativeai as genai

GEMINI_API_KEY = "YOUR_GEMINI_API_KEY_HERE"

genai.configure(api_key=GEMINI_API_KEY)

GEMINI_MODEL = "gemini-1.5-flash"

def call_gemini(prompt: str) -> str:
    model = genai.GenerativeModel(GEMINI_MODEL)
    resp = model.generate_content(prompt)
    try:
        return resp.text
    except Exception:
        return str(resp)


In [ ]:
def build_context(retrieved):
    parts = []
    for i, r in enumerate(retrieved):
        head = f"[Doc {i+1} | score={r['score']:.3f} | id={r['complaint_id']}]"
        parts.append(head + "\n" + r["text"])
    return "\n\n".join(parts)

def rag_answer(query: str, top_k: int = 5):
    retrieved = search(query, top_k)
    context = build_context(retrieved)

    prompt = f"""
User message:
\"\"\"{query}\"\"\"

Here are similar past consumer complaints:
{context}

Tasks:
1. Classify the message as SCAM / NOT_SCAM / UNCERTAIN.
2. Give a 2-4 sentence explanation.
3. Reference helpful docs using "Doc 2", etc.
Do NOT hallucinate facts not supported by the text.
"""

    answer = call_gemini(prompt)
    return answer, retrieved

Put query here

In [ ]:
query = "A recruiter sent me a check and asked me to buy equipment and send back leftover money."

answer, retrieved_docs = rag_answer(query, top_k=5)

print("=== Gemini RAG Answer ===\n")
print(answer)

print("\n=== Retrieved Docs ===\n")
for i, r in enumerate(retrieved_docs, start=1):
    print(f"[Doc {i} | score={r['score']:.3f} | id={r['complaint_id']}]")
    print(r["text"][:350].replace("\n", " "), "...\n")